In [1]:
pip install gradio pandas matplotlib requests


   ---------------------------------------- 0.0/59.7 MB ? eta -:--:--
   -- ------------------------------------- 3.9/59.7 MB 19.9 MB/s eta 0:00:03
   ---- ----------------------------------- 6.6/59.7 MB 16.3 MB/s eta 0:00:04
   ------ --------------------------------- 9.4/59.7 MB 15.1 MB/s eta 0:00:04
   -------- ------------------------------- 12.1/59.7 MB 14.5 MB/s eta 0:00:04
   --------- ------------------------------ 13.9/59.7 MB 13.2 MB/s eta 0:00:04
   ----------- ---------------------------- 17.0/59.7 MB 13.7 MB/s eta 0:00:04
   ------------- -------------------------- 19.9/59.7 MB 13.6 MB/s eta 0:00:03
   --------------- ------------------------ 22.5/59.7 MB 13.5 MB/s eta 0:00:03
   ---------------- ----------------------- 25.2/59.7 MB 13.4 MB/s eta 0:00:03
   ------------------ --------------------- 27.8/59.7 MB 13.4 MB/s eta 0:00:03
   -------------------- ------------------- 30.4/59.7 MB 13.3 MB/s eta 0:00:03
   ---------------------- ----------------- 33.0/59.7 MB 13.3 MB

In [3]:
import gradio as gr
import pandas as pd
import requests
import matplotlib.pyplot as plt
import os
import smtplib
from email.message import EmailMessage

SERPAPI_API_KEY = "1d9172514cebdb6f1bca5c42357edcfdba5a56458fdb93b0cc5aa2d423241f58"  # 🔐 Replace with your SERPAPI key
ALERT_EMAIL = "aniruddh433@gmail.com"  # 📨 Alert recipient
EMAIL_SENDER = "aniruddh433@gmail.com"  # 📤 Sender
EMAIL_PASSWORD = "mddk xhov dnqv pmkn"  # 🛡️ Use App password for Gmail

previous_ranks = {}

def check_keyword_rank(keyword, domain):
    params = {
        "engine": "google",
        "q": keyword,
        "api_key": SERPAPI_API_KEY,
        "num": "100",
    }
    response = requests.get("https://serpapi.com/search", params=params)
    data = response.json()
    organic_results = data.get("organic_results", [])

    for result in organic_results:
        if domain in result.get("link", ""):
            return result.get("position", 0), result.get("link")
    return 0, "Not Found"

def process_input(keywords_list, domain):
    global previous_ranks
    results = []

    for keyword in keywords_list:
        rank, url = check_keyword_rank(keyword, domain)
        previous = previous_ranks.get(keyword, None)
        change = "New" if previous is None else ("Improved" if rank < previous else ("Dropped" if rank > previous else "No Change"))
        previous_ranks[keyword] = rank

        # Send alert if change detected
        if previous is not None and rank != previous:
            send_email_alert(keyword, domain, rank, previous, change)

        results.append({
    "Keyword": keyword,
    "Rank": rank,
    "Ranking URL": url,
    "Previous": previous,
    "Change": change
})

    df = pd.DataFrame(results)
    return df, create_chart(df)

def create_chart(df):
    import matplotlib.pyplot as plt
    from matplotlib.patches import Patch

    plt.figure(figsize=(10, len(df) * 0.6))

    # Sort by rank
    df_sorted = df.sort_values("Rank", ascending=False)

    colors = []
    for change in df_sorted['Change']:
        if change == 'Improved':
            colors.append('green')
        elif change == 'Dropped':
            colors.append('red')
        elif change == 'No Change':
            colors.append('gray')
        else:
            colors.append('blue')  # New

    bars = plt.barh(df_sorted['Keyword'], df_sorted['Rank'], color=colors)

    for bar, rank in zip(bars, df_sorted['Rank']):
        plt.text(rank + 1, bar.get_y() + bar.get_height() / 2, f'Rank {rank}', va='center', fontsize=9)

    plt.xlabel('Google Rank (Lower is Better)', fontsize=11)
    plt.ylabel('Keyword', fontsize=11)
    plt.title('Keyword Ranking Overview', fontsize=14)
    plt.gca().invert_xaxis()  # Rank 1 on left
    plt.grid(axis='x', linestyle='--', alpha=0.6)

    # Legend
    legend_elements = [
        Patch(facecolor='green', label='Improved'),
        Patch(facecolor='red', label='Dropped'),
        Patch(facecolor='gray', label='No Change'),
        Patch(facecolor='blue', label='New'),
    ]
    plt.legend(handles=legend_elements, loc='lower right')

    plt.tight_layout()
    chart_path = "ranking_chart.png"
    plt.savefig(chart_path)
    return chart_path

def send_email_alert(keyword, domain, current, previous, status):
    msg = EmailMessage()
    msg['Subject'] = f"Keyword '{keyword}' - Rank {status}"
    msg['From'] = EMAIL_SENDER
    msg['To'] = ALERT_EMAIL
    msg.set_content(f"Keyword: {keyword}\nDomain: {domain}\nPrevious Rank: {previous}\nCurrent Rank: {current}\nStatus: {status}")

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(EMAIL_SENDER, EMAIL_PASSWORD)
            smtp.send_message(msg)
    except Exception as e:
        print(f"Email failed: {e}")

def main(domain, keyword_text, file):
    keywords = []
    if file is not None:
        df = pd.read_csv(file.name)
        keywords = df.iloc[:, 0].tolist()
    elif keyword_text:
        keywords = [kw.strip() for kw in keyword_text.split(",") if kw.strip()]
    else:
        return "Please input keywords or upload a CSV.", None

    result_df, chart_path = process_input(keywords, domain)
    return result_df, chart_path

demo = gr.Interface(
    fn=main,
    inputs=[
        gr.Textbox(label="Your Domain (e.g., example.com)"),
        gr.Textbox(label="Enter Keywords (comma-separated)"),
        gr.File(label="Upload CSV (1st column = keywords)", file_types=[".csv"])
    ],
    outputs=[
        gr.Dataframe(label="SERP Results"),
        gr.Image(label="Ranking Chart")
    ],
    title="📊 SERP Keyword Rank Checker",
    description="Check Google ranking of keywords for your domain using SERPAPI, with charts & alerts."
)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://91bff18dcf8655723f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
